# Uber Dataset - Cleaning

## 1. Import libraries and load data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
df = pd.read_csv('../data/ncr_ride_bookings_raw.csv')

# Initial inspection
print(df.info())
print(df.head())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Customer  1050

## 2. Handle data type issues

In [2]:
# Remove quotes from string columns
str_cols = df.select_dtypes(include=['object']).columns # Identify string columns
for col in str_cols:
    df[col] = df[col].astype(str).str.replace('"', '', regex=False) # Remove quotes

# Convert date/time columns to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Create a combined datetime column
df['Booking_Datetime'] = pd.to_datetime(
    df['Date'].astype(str) + ' ' + df['Time'].astype(str)
)

# Extract additional temporal features
df['Year'] = df['Booking_Datetime'].dt.year
df['Month'] = df['Booking_Datetime'].dt.month
df['Month_Name'] = df['Booking_Datetime'].dt.month_name()
df['Day'] = df['Booking_Datetime'].dt.day
df['Day_of_Week'] = df['Booking_Datetime'].dt.day_name()
df['Hour'] = df['Booking_Datetime'].dt.hour

# Create buckets for analysis
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'
    
df['Time_of_Day'] = df['Hour'].apply(categorize_time_of_day)

# Convert categorical columns to category dtype, optimizing memory usage
categorical_cols = [
    'Booking Status', 
    'Vehicle Type',
    'Pickup Location',
    'Drop Location',
    'Reason for cancelling by Customer',
    'Driver Cancellation Reason',
    'Incomplete Rides Reason',
    'Payment Method',
    'Month_Name',
    'Day_of_Week',
    'Time_of_Day'
]

for col in categorical_cols:
    df[col] = df[col].astype('category')

# Convert numerical columns to appropriate dtypes, handling errors
numerical_cols = ['Avg VTAT', 'Avg CTAT', 'Booking Value', 'Ride Distance', 
                'Driver Ratings', 'Customer Rating', 'Cancelled Rides by Customer',
                'Cancelled Rides by Driver', 'Incomplete Rides']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce') # Convert to numeric, coercing errors to NaN

# Final inspection
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Date                               150000 non-null  datetime64[ns]
 1   Time                               150000 non-null  object        
 2   Booking ID                         150000 non-null  object        
 3   Booking Status                     150000 non-null  category      
 4   Customer ID                        150000 non-null  object        
 5   Vehicle Type                       150000 non-null  category      
 6   Pickup Location                    150000 non-null  category      
 7   Drop Location                      150000 non-null  category      
 8   Avg VTAT                           139500 non-null  float64       
 9   Avg CTAT                           102000 non-null  float64       
 10  Cancelled Rides by C

## 3. Handle missing values

In [7]:
# Create a flag for incomplete data
df['Has_Missing_Values'] = df.isnull().any(axis=1)

# Document missing value patterns
missing_value_summary = df.isnull().sum()
print(missing_value_summary[missing_value_summary > 0])

# Handle nulls based on booking status logic
# - No Driver Found: Expected to have nulls in VTAT, CTAT, ratings, etc.
# - Cancelled Rides: May have nulls in completion metrics
# - Incomplete Rides: Should have nulls in completion metrics; reason should be filled
# - Completed Rides: Should have minimal nulls; investigate any present

# Create completeness flags
df['Is_Completed'] = df['Booking Status'] == 'Completed'
df['Is_Cancelled'] = df['Booking Status'].str.contains('Cancelled', case=False, na=False)
df['Is_Incomplete'] = df['Booking Status'] == 'Incomplete'
df['No_Driver_Found'] = df['Booking Status'] == 'No Driver Found'

# Fill missing reasons for incomplete rides and cancellations
reason_cols = [
    'Incomplete Rides Reason',
    'Driver Cancellation Reason',
    'Reason for cancelling by Customer'
]

# Ensure 'Reason Not Provided' category exists
for col in reason_cols:
    if isinstance(df[col].dtype, pd.CategoricalDtype):
        if 'Reason Not Provided' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories(['Reason Not Provided'])


# Fill missing incomplete ride reasons
df.loc[df['Is_Incomplete'] & df['Incomplete Rides Reason'].isnull(),
       'Incomplete Rides Reason'] = 'Reason Not Provided'

df.loc[df['Is_Cancelled'] & df['Driver Cancellation Reason'].isnull(),
       'Driver Cancellation Reason'] = 'Reason Not Provided'

df.loc[df['Is_Cancelled'] & df['Reason for cancelling by Customer'].isnull(),
       'Reason for cancelling by Customer'] = 'Reason Not Provided'

Avg VTAT                        10500
Avg CTAT                        48000
Cancelled Rides by Customer    139500
Cancelled Rides by Driver      123000
Incomplete Rides               141000
Booking Value                   48000
Ride Distance                   48000
Driver Ratings                  57000
Customer Rating                 57000
Revenue_per_KM                  48000
Total_TAT                       48000
Rating_Difference               57000
dtype: int64


## 4. Create Calculated Fields

In [4]:
# Revenue Metrics
df['Revenue_per_KM'] = df['Booking Value'] / df['Ride Distance']
df['Revenue_per_KM'] = df['Revenue_per_KM'].replace([np.inf, -np.inf], np.nan)  # Handle division by zero

# Efficiency metrics
df['Total_TAT'] = df['Avg VTAT'] + df['Avg CTAT']

# Rating difference (customer satisfaction vs driver performance)
df['Rating_Difference'] = df['Customer Rating'] - df['Driver Ratings']

# Distance categories
def categorize_distance(distance):
    if pd.isnull(distance):
        return 'Unknown'
    elif distance < 5:
        return 'Short'
    elif 5 <= distance < 15:
        return 'Medium'
    else:
        return 'Long'
    
df['Distance_Category'] = df['Ride Distance'].apply(categorize_distance)

# Value categories
def categorize_value(value):
    if pd.isnull(value):
        return 'Unknown'
    elif value < 100:
        return 'Low'
    elif 100 <= value < 500:
        return 'Medium'
    else:
        return 'High'
    
df['Value_Category'] = df['Booking Value'].apply(categorize_value)


## 5. Data Quality Checks

In [5]:
# Check for duplicate booking IDs
duplicate_booking_id = df.duplicated(subset=['Booking ID'], keep=False)
print(f"Number of duplicate booking IDs found: {duplicate_booking_id.sum()}")

# Check for logical inconsistencies (E.g., Completed rides with null VTAT/CTAT)
inconsistent_completed_rides = df[
    (df['Is_Completed']) & 
    (df['Avg VTAT'].isnull() | df['Avg CTAT'].isnull())
]
print(f"Number of inconsistent completed rides found: {len(inconsistent_completed_rides)}")

# Validate calculated fields
invalid_revenue_per_km = df[
    (df['Revenue_per_KM'] < 0) |
    (df['Revenue_per_KM'].isnull() & df['Booking Value'].notnull() & df['Ride Distance'].notnull())
]
print(f"Number of invalid Revenue_per_KM entries found: {len(invalid_revenue_per_km)}")

# Validate ratings are within expected range (0-5) 
# Driver Ratings
invalid_driver_ratings = df[
    (df['Driver Ratings'] < 0) | (df['Driver Ratings'] > 5)
]
print(f"Number of invalid Driver Ratings entries found: {len(invalid_driver_ratings)}")
# Customer Ratings
invalid_customer_ratings = df[
    (df['Customer Rating'] < 0) | (df['Customer Rating'] > 5)
]
print(f"Number of invalid Customer Ratings entries found: {len(invalid_customer_ratings)}")

# Check for negative values where they shouldn't be
negative_checks = ['Booking Value', 'Ride Distance', 'Avg VTAT', 'Avg CTAT']
for col in negative_checks:
    negatives = df[df[col] < 0]
    print(f"Number of negative entries in {col}: {len(negatives)}")

Number of duplicate booking IDs found: 2457
Number of inconsistent completed rides found: 0
Number of invalid Revenue_per_KM entries found: 0
Number of invalid Driver Ratings entries found: 0
Number of invalid Customer Ratings entries found: 0
Number of negative entries in Booking Value: 0
Number of negative entries in Ride Distance: 0
Number of negative entries in Avg VTAT: 0
Number of negative entries in Avg CTAT: 0


## 6. Export Cleaned Data

In [6]:
df.to_csv('../data/uber_dataset_cleaned.csv', index=False)

# Create a version for PostgreSQL (with proper formatting)
df.to_csv('../data/uber_dataset_postgresql.csv', index=False, date_format='%Y-%m-%d')

print("Cleaned data exported successfully.")
print(f"Original rows: {len(df)}")
print(f"Cleaned rows: {len(df)}")
print(f"Columns added: {len(df.columns) - len(pd.read_csv('../data/ncr_ride_bookings_raw.csv').columns)}")

Cleaned data exported successfully.
Original rows: 150000
Cleaned rows: 150000
Columns added: 18
